In [1]:
!pip install gliner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 KB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 57.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 KB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 KB 56.6 MB/s eta 0:00:00


In [3]:
import pandas as pd
from gliner import GLiNER
from tqdm import tqdm


In [4]:
df=pd.read_csv("titles.csv",on_bad_lines='skip')
df.head()

,year,id,title,journal_name,col5
0,2024,PMC11197899,Bilateral Optic Neuroretinitis: Uncommon Compl...,Neuroophthalmology,"""Case Reports;Journal Article;"""
1,2024,PMC11095573,Bilateral Optic Neuropathy Associated with Acu...,Neuroophthalmology,"""Case Reports;"""
2,2024,PMC10899702,Telemedicine in patients with obsessive-compul...,Front Hum Neurosci,"""Case Reports;Journal Article;"""
3,2024,PMC11073879,Compulsive Repetitive Flexion With Breath-Hold...,Neurol Clin Pract,"""Case Reports;"""
4,2024,PMC10864136,Spontaneous Regression of Glioma-Mimicking Bra...,Brain Tumor Res Treat,"""Case Reports;Journal Article;"""


In [5]:
df.shape

(9343, 5)

In [6]:
gliner_label_descriptions={
    "gene":"This domain includes gene identifiers and markers like 0610007f13rik, 0610008b10rik, and 0610010i13rik commonly used in genomic research." ,
    "taxonomy": "This domain includes species names and taxonomy-related terms such as aardvark, abalone, abies, and abiotrophia.", 
    "brain_atlas": "This domain includes brain region codes used in anatomical brain atlases like 10cb, 10m, 10pp, and 10v." , 
    "anatomy": "This domain includes anatomical and structural identifiers like 3nf, 4nf, a10a, and a42 used in biomedical literature.",
    "molecule": "This domain includes molecular identifiers and chemical codes such as 0000548579903, 0113428913, 10000347, and 10008747.",
    "disease": "This domain includes disease names and conditions such as abetalipoproteinemia, abscess, abortion, and acanthocephaliasis.",
    "metadata": "This domain includes metadata terms and identifiers used in biomedical datasets and genomic annotations, such as chromosomal loci (e.g., 10q112, 10q21), ontology-based IDs (e.g., IAO, BIRNLex, ILX terms), and structured descriptors of experimental protocols, subjects, and datasets. These terms typically describe contextual or referential information rather than biological entities themselves.",
    "protein": "This domain includes protein names and identifiers like 11b6, 15e11, 17betahsd11, and 1a13b relevant to biochemical studies.",
    "phenotype": "This domain includes phenotype-related terms and conditions like abolished, acidemia, acidophilic, and actinomorphic." ,
    "cognitive":"This domain includes cognitive science and psychological terms such as altruism, anticipation, attitude, and belief.",
    "medical_procedures":"This domain includes diagnostic, surgical, and therapeutic procedures performed for clinical evaluation or intervention. Examples include bronchoscopy, deep brain stimulation, coronary angioplasty, endoscopy, MRI scan, lumbar puncture, and surgical resection. These terms represent specific medical or laboratory actions performed on patients to diagnose, monitor, or treat medical conditions.",
    "treatment": "This domain includes therapeutic interventions and treatment strategies such as chemotherapy, insulin therapy, physical rehabilitation, and cognitive behavioral therapy. These terms describe methods used to manage, alleviate, or cure medical conditions."
}

In [7]:
from gliner import GLiNER

model = GLiNER.from_pretrained("Ihor/gliner-biomed-large-v1.0")

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

gliner_config.json: 0.00B [00:00, ?B/s]

.gitattributes: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/970 [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/1.91G [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

trainer_state.json: 0.00B [00:00, ?B/s]

In [8]:
# Function used to map domains to the text given
def classify_paper(text, gliner_model, gliner_label_descriptions):
    output = gliner_model.predict_entities(text, gliner_label_descriptions)
    predicted_labels = list({ent["label"] for ent in output})
    return predicted_labels

In [9]:
# function used to see which entity has been mapped to which domain
def highlight_entities(text, entities):
    highlighted = ""
    last_idx = 0
    for ent in sorted(entities, key=lambda x: x['start']):
        highlighted += text[last_idx:ent['start']]
        highlighted += f"[{ent['text']}]({ent['label']})"
        last_idx = ent['end']
    highlighted += text[last_idx:]
    return highlighted

In [10]:
import re
import html
import pandas as pd
import numpy as np

# Function to clean the abstracts from the html tags

def clean_abstract(text):
    if pd.isna(text) or text is np.nan:
        return ""

    # Unescape HTML entities (if any)
    text = html.unescape(text)

    # Replace <h> and <p> tags with newlines (optional: just space if you prefer)
    text = re.sub(r'</?(h|p)>', '\n', text, flags=re.IGNORECASE)

    # Remove other HTML tags if present
    text = re.sub(r'<.*?>', '', text)

    # Fix common encoding issues
    text = text.replace("â\x80\x93", "–").replace("â\x80\x94", "—").replace("â\x80\x9c", '"').replace("â\x80\x9d", '"')
    text = text.replace("â\x80\x98", "'").replace("â\x80\x99", "'")

    # Normalize whitespace
    text = re.sub(r'\s+', ' ', text)

    # Final trim
    return text.strip()


In [11]:
test_text = "A case of chorea-acanthocytosis with significant improvement of symptoms at one year with deep brain stimulation: case report and literature review"
predicted = classify_paper(test_text, model, gliner_label_descriptions)
print(predicted)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


['disease', 'phenotype', 'metadata', 'treatment']


In [12]:
# Mapping domains to the titles
results = []
for _, row in tqdm(df.iterrows(), total=len(df)):
    text = f"{row['title']}"
    predicted = classify_paper(text, model, gliner_label_descriptions)  
    results.append(predicted)

df["predicted_domains"] = results


100%|███████████████████████████████████████████████████████████████████████| 9343/9343 [39:19<00:00,  3.96it/s]


In [13]:
df.to_csv("papers_with_title_domains.csv", index=False)

In [14]:
print("Titles -> Predicted Domains")

for i in range(20):
    title = df.loc[i, 'title']
    predicted = str(df.loc[i, 'predicted_domains'])
    print(f"{title} -> {predicted}")


Titles -> Predicted Domains
Bilateral Optic Neuroretinitis: Uncommon Complication of COVID-19. -> ['disease']
Bilateral Optic Neuropathy Associated with Acute Inhaled Marijuana Use: Case Report and Review of the Literature. -> ['disease', 'molecule']
Telemedicine in patients with obsessive-compulsive disorder after deep brain stimulation: a case series. -> ['disease', 'medical_procedures', 'treatment']
Compulsive Repetitive Flexion With Breath-Holding in Sagging Brain Syndrome. -> ['phenotype', 'disease']
Spontaneous Regression of Glioma-Mimicking Brainstem Lesion in a Child: A Case Report. -> ['disease', 'metadata', 'treatment']
Combined central and peripheral demyelination: a case report resembling encephalomyeloradiculoneuropathy. -> ['disease', 'anatomy']
Case report: Headache as the sole neurological symptom in autoimmune glial fibrillary acidic protein (GFAP) astrocytopathy. -> ['phenotype', 'disease', 'protein']
Case report: Neuronal intranuclear inclusion disease initially mimi

In [15]:
title = "NeuroBehcet's-related intracranial hypertension without cerebral venous thrombosis: case report and review of literature."
entities = model.predict_entities(title, gliner_label_descriptions)  
print(highlight_entities(title, entities))


[NeuroBehcet](brain_atlas)'s-related [intracranial hypertension](disease) without [cerebral venous thrombosis](disease): [case report](metadata) and review of literature.


In [16]:
for i in range(10):
    title=df.loc[i,'title']
    entities = model.predict_entities(title, gliner_label_descriptions)  
    print(highlight_entities(title, entities))

[Bilateral Optic Neuroretinitis](disease): Uncommon Complication of [COVID-19](disease).
[Bilateral Optic Neuropathy](disease) Associated with Acute Inhaled [Marijuana](molecule) Use: Case Report and Review of the Literature.
[Telemedicine](medical_procedures) in patients with [obsessive-compulsive disorder](disease) after [deep brain stimulation](treatment): a case series.
[Compulsive Repetitive Flexion](phenotype) With [Breath-Holding](phenotype) in [Sagging Brain Syndrome](disease).
[Spontaneous Regression](treatment) of [Glioma-Mimicking Brainstem Lesion](disease) in a Child: A [Case Report](metadata).
Combined [central](anatomy) and [peripheral](anatomy) demyelination: a case report resembling [encephalomyeloradiculoneuropathy](disease).
Case report: [Headache](phenotype) as the sole [neurological symptom](phenotype) in autoimmune [glial fibrillary acidic protein](protein) ([GFAP](protein)) [astrocytopathy](disease).
Case report: [Neuronal intranuclear inclusion disease](disease) 

In [17]:
df.head()

,year,id,title,journal_name,col5,predicted_domains
0,2024,PMC11197899,Bilateral Optic Neuroretinitis: Uncommon Compl...,Neuroophthalmology,"""Case Reports;Journal Article;""",[disease]
1,2024,PMC11095573,Bilateral Optic Neuropathy Associated with Acu...,Neuroophthalmology,"""Case Reports;""","[disease, molecule]"
2,2024,PMC10899702,Telemedicine in patients with obsessive-compul...,Front Hum Neurosci,"""Case Reports;Journal Article;""","[disease, medical_procedures, treatment]"
3,2024,PMC11073879,Compulsive Repetitive Flexion With Breath-Hold...,Neurol Clin Pract,"""Case Reports;""","[phenotype, disease]"
4,2024,PMC10864136,Spontaneous Regression of Glioma-Mimicking Bra...,Brain Tumor Res Treat,"""Case Reports;Journal Article;""","[disease, metadata, treatment]"


In [18]:
dfa=pd.read_csv("titles_with_abstracts.csv",on_bad_lines='skip')
dfa.head()

,year,id,title,journal_name,col5,abstract
0,2024,PMC11197899,Bilateral Optic Neuroretinitis: Uncommon Compl...,Neuroophthalmology,"""Case Reports;Journal Article;""","<h>ABSTRACT<h><p>At the end of 2019, the sever..."
1,2024,PMC11095573,Bilateral Optic Neuropathy Associated with Acu...,Neuroophthalmology,"""Case Reports;""",NaN
2,2024,PMC10899702,Telemedicine in patients with obsessive-compul...,Front Hum Neurosci,"""Case Reports;Journal Article;""",<h>Background<h><p>Patients suffering from ref...
3,2024,PMC11073879,Compulsive Repetitive Flexion With Breath-Hold...,Neurol Clin Pract,"""Case Reports;""",<h>Background and Objective<h><p>Spontaneous i...
4,2024,PMC10864136,Spontaneous Regression of Glioma-Mimicking Bra...,Brain Tumor Res Treat,"""Case Reports;Journal Article;""",NaN


In [19]:
dfa['abstract'].head()

0    <h>ABSTRACT<h><p>At the end of 2019, the sever...
1                                                  NaN
2    <h>Background<h><p>Patients suffering from ref...
3    <h>Background and Objective<h><p>Spontaneous i...
4                                                  NaN
Name: abstract, dtype: object

In [20]:
dfa['abstract'].tolist()

['<h>ABSTRACT<h><p>At the end of 2019, the severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2) emerged in Wuhan, China. On 11 March 2020, the World Health Organization declared COVID-19 a pandemic. This virus affects many organs, including the eye, and can manifest through various clinical manifestations.<p><p>Multiple neuro-ophthalmological manifestations have been reported in association with COVID-19, including, Optic Neuritis, cranial nerve palsies, eye movement abnormalities, and visual field defects. In this article, we report a case of bilateral neuroretinitis in association with (SARS-CoV-2).<p>',
 nan,
 '<h>Background<h><p>Patients suffering from refractory obsessive-compulsive disorder (OCD) who have undergone deep brain stimulation (DBS) surgery require repeated in-person programming visits. These sessions could be labor-intensive and may not always be feasible, particularly when in-person hospital visits are restricted. Telemedicine is emerging as a potential suppl

In [21]:
dfa['cleaned_abstract'] = dfa['abstract'].apply(clean_abstract)

In [22]:
dfa['cleaned_abstract'].tolist()

['ABSTRACT At the end of 2019, the severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2) emerged in Wuhan, China. On 11 March 2020, the World Health Organization declared COVID-19 a pandemic. This virus affects many organs, including the eye, and can manifest through various clinical manifestations. Multiple neuro-ophthalmological manifestations have been reported in association with COVID-19, including, Optic Neuritis, cranial nerve palsies, eye movement abnormalities, and visual field defects. In this article, we report a case of bilateral neuroretinitis in association with (SARS-CoV-2).',
 '',
 'Background Patients suffering from refractory obsessive-compulsive disorder (OCD) who have undergone deep brain stimulation (DBS) surgery require repeated in-person programming visits. These sessions could be labor-intensive and may not always be feasible, particularly when in-person hospital visits are restricted. Telemedicine is emerging as a potential supplementary tool for post-op

In [23]:
dfa.drop('abstract', axis=1, inplace=True)

In [24]:
dfa = dfa.rename(columns={'cleaned_abstract': 'abstract'})

In [25]:
dfa.head()

,year,id,title,journal_name,col5,abstract
0,2024,PMC11197899,Bilateral Optic Neuroretinitis: Uncommon Compl...,Neuroophthalmology,"""Case Reports;Journal Article;""","ABSTRACT At the end of 2019, the severe acute ..."
1,2024,PMC11095573,Bilateral Optic Neuropathy Associated with Acu...,Neuroophthalmology,"""Case Reports;""",
2,2024,PMC10899702,Telemedicine in patients with obsessive-compul...,Front Hum Neurosci,"""Case Reports;Journal Article;""",Background Patients suffering from refractory ...
3,2024,PMC11073879,Compulsive Repetitive Flexion With Breath-Hold...,Neurol Clin Pract,"""Case Reports;""",Background and Objective Spontaneous intracran...
4,2024,PMC10864136,Spontaneous Regression of Glioma-Mimicking Bra...,Brain Tumor Res Treat,"""Case Reports;Journal Article;""",


In [26]:
dfa['abstract'].isna().sum()

np.int64(0)

In [27]:
dfa = dfa[dfa['abstract'].notna() & (dfa['abstract'].str.strip() != "")]
dfa.reset_index(drop=True, inplace=True)

In [28]:
dfa.head()

,year,id,title,journal_name,col5,abstract
0,2024,PMC11197899,Bilateral Optic Neuroretinitis: Uncommon Compl...,Neuroophthalmology,"""Case Reports;Journal Article;""","ABSTRACT At the end of 2019, the severe acute ..."
1,2024,PMC10899702,Telemedicine in patients with obsessive-compul...,Front Hum Neurosci,"""Case Reports;Journal Article;""",Background Patients suffering from refractory ...
2,2024,PMC11073879,Compulsive Repetitive Flexion With Breath-Hold...,Neurol Clin Pract,"""Case Reports;""",Background and Objective Spontaneous intracran...
3,2024,PMC10825037,Combined central and peripheral demyelination:...,Front Neurol,"""Case Reports;Journal Article;""",Combined central and peripheral demyelination ...
4,2024,PMC11063300,Case report: Headache as the sole neurological...,Front Neurol,"""Case Reports;Journal Article;""",Autoimmune glial fibrillary acidic protein (GF...


In [29]:
dfa.shape

(8568, 6)

In [30]:
# Check for duplicate paper_ids in your title-predicted domains dataframe
duplicate_ids = df['id'][df['id'].duplicated()]
print("Duplicate paper_ids:", duplicate_ids.unique())
print("How many duplicates:", len(duplicate_ids))


Duplicate paper_ids: ['PMC10825037' 'PMC10874322' 'PMC11021265' 'PMC10951697' 'PMC11444322'
 'PMC10764086' 'PMC10797035' 'PMC10802081' 'PMC11040634' 'PMC10799339'
 'PMC10790611' 'PMC11531025' 'PMC10834122' 'PMC11138328' 'PMC10850352'
 'PMC11235873' 'PMC10863564' 'PMC10858109' 'PMC10790605' 'PMC10034697'
 'PMC9944315' 'PMC11310725' 'PMC10588606' 'PMC11045180' 'PMC10369111'
 'PMC9988330' 'PMC10174133' 'PMC9892895' 'PMC10091426' 'PMC10091440'
 'PMC9928457' 'PMC9944787' 'PMC9868692' 'PMC9845570' 'PMC9877408'
 'PMC9842449' 'PMC10009412' 'PMC9945400' 'PMC9871926' 'PMC9893163'
 'PMC9779985' 'PMC9845611' 'PMC9945291' 'PMC8689546' 'PMC9064752'
 'PMC9103496' 'PMC8810698' 'PMC8770749' 'PMC8826287' 'PMC8903752'
 'PMC8813978' 'PMC9103631' 'PMC8810502' 'PMC9291677' 'PMC9244739'
 'PMC10311077' 'PMC8364776' 'PMC9103260' 'PMC8820373' 'PMC9833462'
 'PMC8566116' 'PMC8794246' 'PMC8903770' 'PMC8879509' 'PMC8826293'
 'PMC8954303' 'PMC8931735' 'PMC8689557' 'PMC9291713' 'PMC8792598'
 'PMC8831329' 'PMC8689528'

In [31]:
# Drop duplicates (keep first occurrence or handle manually if needed)
df = df.drop_duplicates(subset='id')

In [32]:
# Check for duplicate paper_ids in your title-predicted domains dataframe
duplicate_ids = df['id'][df['id'].duplicated()]
print("Duplicate paper_ids:", duplicate_ids.unique())
print("How many duplicates:", len(duplicate_ids))


Duplicate paper_ids: []
How many duplicates: 0


In [33]:
dfa = dfa.merge(df[['id', 'predicted_domains']], on='id', how='inner')

In [34]:
dfa.shape

(8568, 7)

In [35]:
dfa.head()

,year,id,title,journal_name,col5,abstract,predicted_domains
0,2024,PMC11197899,Bilateral Optic Neuroretinitis: Uncommon Compl...,Neuroophthalmology,"""Case Reports;Journal Article;""","ABSTRACT At the end of 2019, the severe acute ...",[disease]
1,2024,PMC10899702,Telemedicine in patients with obsessive-compul...,Front Hum Neurosci,"""Case Reports;Journal Article;""",Background Patients suffering from refractory ...,"[disease, medical_procedures, treatment]"
2,2024,PMC11073879,Compulsive Repetitive Flexion With Breath-Hold...,Neurol Clin Pract,"""Case Reports;""",Background and Objective Spontaneous intracran...,"[phenotype, disease]"
3,2024,PMC10825037,Combined central and peripheral demyelination:...,Front Neurol,"""Case Reports;Journal Article;""",Combined central and peripheral demyelination ...,"[disease, anatomy]"
4,2024,PMC11063300,Case report: Headache as the sole neurological...,Front Neurol,"""Case Reports;Journal Article;""",Autoimmune glial fibrillary acidic protein (GF...,"[phenotype, disease, protein]"


In [36]:
dfa = dfa.rename(columns={'predicted_domains': 'title_domains'})
dfa.head()

,year,id,title,journal_name,col5,abstract,title_domains
0,2024,PMC11197899,Bilateral Optic Neuroretinitis: Uncommon Compl...,Neuroophthalmology,"""Case Reports;Journal Article;""","ABSTRACT At the end of 2019, the severe acute ...",[disease]
1,2024,PMC10899702,Telemedicine in patients with obsessive-compul...,Front Hum Neurosci,"""Case Reports;Journal Article;""",Background Patients suffering from refractory ...,"[disease, medical_procedures, treatment]"
2,2024,PMC11073879,Compulsive Repetitive Flexion With Breath-Hold...,Neurol Clin Pract,"""Case Reports;""",Background and Objective Spontaneous intracran...,"[phenotype, disease]"
3,2024,PMC10825037,Combined central and peripheral demyelination:...,Front Neurol,"""Case Reports;Journal Article;""",Combined central and peripheral demyelination ...,"[disease, anatomy]"
4,2024,PMC11063300,Case report: Headache as the sole neurological...,Front Neurol,"""Case Reports;Journal Article;""",Autoimmune glial fibrillary acidic protein (GF...,"[phenotype, disease, protein]"


In [37]:
# Mapping domains to the abstracts
results = []
for _, row in tqdm(dfa.iterrows(), total=len(dfa)):
    text = f"{row['abstract']}"
    predicted = classify_paper(text, model, gliner_label_descriptions)  
    results.append(predicted)

dfa["abstract_domains"] = results


100%|█████████████████████████████████████████████████████████████████████| 8568/8568 [1:06:30<00:00,  2.15it/s]


In [38]:
dfa.to_csv("abstracts_titles_wdomains.csv",index=False)

In [39]:
dfa['abstract_domains'].tolist()

[['disease', 'anatomy', 'molecule', 'phenotype'],
 ['treatment', 'phenotype', 'cognitive', 'disease', 'medical_procedures'],
 ['metadata', 'phenotype', 'cognitive', 'disease', 'medical_procedures'],
 ['treatment', 'phenotype', 'anatomy', 'disease', 'medical_procedures'],
 ['treatment',
  'protein',
  'phenotype',
  'anatomy',
  'disease',
  'medical_procedures',
  'brain_atlas',
  'molecule'],
 ['treatment',
  'phenotype',
  'anatomy',
  'cognitive',
  'gene',
  'disease',
  'medical_procedures',
  'molecule'],
 ['phenotype',
  'anatomy',
  'cognitive',
  'gene',
  'disease',
  'medical_procedures',
  'brain_atlas'],
 ['treatment', 'phenotype', 'gene', 'disease', 'medical_procedures'],
 ['taxonomy',
  'treatment',
  'protein',
  'phenotype',
  'anatomy',
  'gene',
  'disease',
  'medical_procedures',
  'molecule'],
 ['treatment', 'phenotype', 'anatomy', 'disease', 'medical_procedures'],
 ['treatment',
  'phenotype',
  'gene',
  'anatomy',
  'disease',
  'medical_procedures'],
 ['phenot

In [40]:
dfa.head()

,year,id,title,journal_name,col5,abstract,title_domains,abstract_domains
0,2024,PMC11197899,Bilateral Optic Neuroretinitis: Uncommon Compl...,Neuroophthalmology,"""Case Reports;Journal Article;""","ABSTRACT At the end of 2019, the severe acute ...",[disease],"[disease, anatomy, molecule, phenotype]"
1,2024,PMC10899702,Telemedicine in patients with obsessive-compul...,Front Hum Neurosci,"""Case Reports;Journal Article;""",Background Patients suffering from refractory ...,"[disease, medical_procedures, treatment]","[treatment, phenotype, cognitive, disease, med..."
2,2024,PMC11073879,Compulsive Repetitive Flexion With Breath-Hold...,Neurol Clin Pract,"""Case Reports;""",Background and Objective Spontaneous intracran...,"[phenotype, disease]","[metadata, phenotype, cognitive, disease, medi..."
3,2024,PMC10825037,Combined central and peripheral demyelination:...,Front Neurol,"""Case Reports;Journal Article;""",Combined central and peripheral demyelination ...,"[disease, anatomy]","[treatment, phenotype, anatomy, disease, medic..."
4,2024,PMC11063300,Case report: Headache as the sole neurological...,Front Neurol,"""Case Reports;Journal Article;""",Autoimmune glial fibrillary acidic protein (GF...,"[phenotype, disease, protein]","[treatment, protein, phenotype, anatomy, disea..."


In [41]:
print("Abstracts -> Predicted Domains")

for i in range(20):
    abst = dfa.loc[i, 'abstract']
    predicted = str(dfa.loc[i, 'abstract_domains'])
    print(f"{abst} -> {predicted}")


Abstracts -> Predicted Domains
ABSTRACT At the end of 2019, the severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2) emerged in Wuhan, China. On 11 March 2020, the World Health Organization declared COVID-19 a pandemic. This virus affects many organs, including the eye, and can manifest through various clinical manifestations. Multiple neuro-ophthalmological manifestations have been reported in association with COVID-19, including, Optic Neuritis, cranial nerve palsies, eye movement abnormalities, and visual field defects. In this article, we report a case of bilateral neuroretinitis in association with (SARS-CoV-2). -> ['disease', 'anatomy', 'molecule', 'phenotype']
Background Patients suffering from refractory obsessive-compulsive disorder (OCD) who have undergone deep brain stimulation (DBS) surgery require repeated in-person programming visits. These sessions could be labor-intensive and may not always be feasible, particularly when in-person hospital visits are restricted.

In [42]:
# to see the entities highlighted with their domains
for i in range(3):
    abst=dfa.loc[i,'abstract']
    entities = model.predict_entities(abst, gliner_label_descriptions)  
    print(highlight_entities(abst, entities))

ABSTRACT At the end of 2019, the [severe acute respiratory syndrome coronavirus 2](molecule) ([SARS-CoV-2](molecule)) emerged in Wuhan, China. On 11 March 2020, the World Health Organization declared [COVID-19](disease) a pandemic. This [virus](molecule) affects many [organs](anatomy), including the [eye](anatomy), and can manifest through various [clinical manifestations](phenotype). Multiple [neuro-ophthalmological manifestations](phenotype) have been reported in association with [COVID-19](disease), including, [Optic Neuritis](disease), [cranial nerve palsies](disease), [eye](anatomy) movement abnormalities, and [visual field defects](disease). In this article, we report a case of [bilateral neuroretinitis](disease) in association with ([SARS-CoV-2](molecule)).
Background Patients suffering from [refractory obsessive-compulsive disorder](disease) ([OCD](disease)) who have undergone [deep brain stimulation (DBS) surgery](medical_procedures) require repeated in-person programming visi

In [43]:
df=pd.read_csv("abstracts_titles_wdomains.csv",on_bad_lines='skip')

In [44]:
df.shape

(8568, 8)

In [45]:
df.columns

Index(['year', 'id', 'title', 'journal_name', 'col5', 'abstract',
       'title_domains', 'abstract_domains'],
      dtype='object')

In [46]:
import pandas as pd
import ast

df['title_domains'] = df['title_domains'].apply(ast.literal_eval)
df['abstract_domains'] = df['abstract_domains'].apply(ast.literal_eval)


In [47]:
df['title_domains'].tolist()

[['disease'],
 ['disease', 'medical_procedures', 'treatment'],
 ['phenotype', 'disease'],
 ['disease', 'anatomy'],
 ['phenotype', 'disease', 'protein'],
 ['disease', 'metadata', 'phenotype', 'medical_procedures'],
 ['disease', 'cognitive'],
 ['disease', 'gene', 'medical_procedures', 'protein'],
 ['disease'],
 ['disease', 'anatomy', 'metadata'],
 ['disease', 'phenotype', 'metadata', 'treatment'],
 ['phenotype', 'anatomy', 'metadata', 'disease'],
 ['disease', 'molecule', 'treatment'],
 ['disease', 'metadata', 'brain_atlas'],
 ['metadata', 'phenotype', 'gene', 'disease', 'molecule'],
 ['disease', 'metadata'],
 ['phenotype', 'anatomy', 'cognitive'],
 ['disease', 'phenotype'],
 ['molecule', 'medical_procedures'],
 ['disease', 'metadata', 'treatment'],
 ['disease'],
 ['disease', 'phenotype', 'metadata'],
 ['disease', 'anatomy'],
 ['treatment', 'anatomy', 'cognitive', 'disease', 'medical_procedures'],
 ['disease', 'anatomy'],
 ['disease', 'medical_procedures', 'treatment'],
 ['taxonomy', 'phe